### Ai Earth Engine 《官方文档》- 应用案例
- 按区域检索与下载Sentine-2 L2A影像
> 哨兵2号（Sentinel-2）是欧洲空间局哥白尼计划下的一个地球观测任务，该任务主要对地球表面进行观测以提供相关遥测服务,如森林监测、土地覆盖变化侦测、天然灾害管理。该计划由2颗相同的卫星哨兵2号A（Sentinel-2A）与B（Sentinel-2B）组成的卫星群。

In [2]:
import aie
# 使用token进行鉴权
aie.Authenticate(token='a1e1845361248013ff7fdf5447a00e38')
# 使用 aliyun access_key_id进行鉴权
# aie.Authenticate(access_key_id="*", access_key_secret="*")
# 使用 aliyun RAM STS 进行鉴权(需要 aiearth-core>=1.0.3)
# aie.Authenticate(access_key_id='*', access_key_secret='*', access_key_security_token='*')
aie.Initialize()

#### 定义矢量区域

In [ ]:
region = aie.FeatureCollection('China_Province') \
            .filter(aie.Filter.eq('province', '浙江省'))             
map = aie.Map(
    center=region.getCenter(),
    height=300,
    zoom=6
)
vis_params = {
    'color': '#00FF00'
}
map.addLayer(
    region,
    vis_params,
    'region',
    bounds=region.getBounds()
)
map

#### Sentinel-2 数据检索

In [8]:
#定义函数，获取指定时间范围内的过滤Sentinel-2影像集合和中值合成的区域裁剪影像
def s2_collection(start_date, end_date):
    s2 = aie.ImageCollection('SENTINEL_MSIL2A') \
            .filterBounds(region) \
            .filterDate(start_date, end_date) \
            .filter('eo:cloud_cover<20')
    mosaic_image = s2.median().clipToCollection(region)
    return s2, mosaic_image
s2, s2_mosaic = s2_collection('2021-04-01', '2022-08-30')

#### 数据可视化

In [9]:
vis_params = {
    'bands': ['B4', 'B3', 'B2'],   
    'min': 0,
    'max': 3500
}
map.addLayer(
    s2_mosaic,
    vis_params,
    'Ture color',
    bounds=region.getBounds()
)
map

Map(bottom=5538.0, center=[29.15771484375, 120.43212890625], controls=(ZoomControl(options=['position', 'zoom_…

#### 影像导出（离线计算）

In [ ]:
# 导出镶嵌影像
task = aie.Export.image.toAsset(s2_mosaic, 's2_mosaic', 200)
task.start()

### Ai Earth Engine 《官方文档》- 应用案例
- Landsat 系列数据去云
> AIE平台中提供的 Landsat 影像均为 USGS Collection 2 中的数据，该集合使用 QA_PIXEL 波段对云、雪进行描述。因此，可通过位运算和逻辑运算，生成云掩膜，利用 updateMask 函数去除可能是云或者云阴影的区域。此算法适用于 Landsat-5 / 7 / 8 / 9 。